In [31]:
import os

from dotenv import dotenv_values
from langchain.memory import ConversationBufferMemory

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

# return_messages=True : Chat 모델인 경우 
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}